## Business problem

Port terminals constantly strive to improve the efficiency of their operations through a careful
management of their berth facilities, machinery and personnel. 



The most important variables when planning terminal operations is knowing which vessels will arrive at the terminal and when.


MarineTraffic aims at being the best visibility providers by providing up to date vessel tracking data (using AIS), as well as additional derived information such as the estimated time of arrival (ETA) of a vessel to a port of interest.


AIS messages contain information on the port that the vessel is traveling to as well as the estimated time of arrival.

However, since ports may consist of more than one terminal, the exact terminal that the
vessel will visit is not known in advance making it difficult for MarineTraffic to assign future arrivals to terminals which, in turn, limits the ability to measure terminal congestion and calculate more accurate terminal arrival times.



A model which predicts the terminal a vessel will travel to has the potential to help all parties involved in a port call to plan their operations more effectively

# Data Description

A dataset has been extracted containing container calls at terminals that took place during the past 3
years for the Port of **Hamburg and Port of Los Angeles**. The dataset contains the following fields

A dataset has been extracted containing container calls at terminals that took place during the past 3
years for the Port of Hamburg and Port of Los Angeles. The dataset contains the following fields;


* **last_port**: Port where the *last* terminal call by vessel is recorded.



* **last_terminal**: The immediately previous *terminal* call of the vessel.


* **last_terminal_doc_timestamp**: Timestamp of previous terminal call.


* **current_port**: Port where the current terminal call by vessel is recorded.


* **current_terminal**: Current terminal call of the vessel.


* **shipname**: Name of the vessel.


* **dock_timestamp**: Timestamp of current terminal call.


* **GRT**: Vessel capacity (gross tonnage unit).


* **TEU**: Vessel capacity (twenty-foot equivalent Unit).


* **length**: Vessel length.


* **width**: Vessel width.

# Goals and Deliverable


The goal of this task is to implement & evaluate the accuracy of a solution that predicts the terminal
that a vessel will call.
**The product team claims that the history of terminals visited by a vessel in the past is a critical factor
that should be incorporated into the model**.

Some important steps that your solution would be expected to address and describe are the following:
* What features have you finally selected and engineered for your modeling approach? What led you to these choices? Why & how have you processed them?
* Which features seem to be the most important & how did you evaluate their importance?
* Do your findings agree with the product team’s insights discussed above? Before developing a ML model, how would you evaluate the importance/predictive power of one of the productidentified features as an independent variable?
* What type of prediction/training model have you chosen and why?
* How well does your predictive solution perform in terms of predicting the terminal a vessel will call?
* What different metrics/graphs can you use in order to understand when & why the algorithm fails/succeeds?
* What would be your baseline (i.e. a “naive” approach) to compare against?

# Solution Procedure

### problem understanding & EDA

In order to understand the problem we have to dive into the data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import RobustScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder

from sklearn.ensemble import RandomForestClassifier

In [2]:
raw_data = pd.read_csv("data/mt_terminal_calls.csv")

In [3]:
raw_data.head(5)

,last_port,last_terminal,last_terminal_doc_timestamp,current_port,current_terminal,shipname,doc_timestamp,grt,teu,length,width
0,BREMERHAVEN,North Sea Terminal,2020-01-02 16:12:00.000,HAMBURG,Eurogate Container Terminal Hamburg,HEINRICH EHLER,2020-01-03 06:20:00.000,17488,1421,168.11,26.80
1,TILBURY,London Container Terminal,2020-01-01 21:56:00.000,HAMBURG,C. Steinweg Multipurpose Terminal,HENNEKE RAMBOW,2020-01-03 07:45:00.000,9981,868,134.40,22.74
2,ROTTERDAM MAASVLAKTE,Rotterdam World Gateway Terminal,2020-01-02 12:06:00.000,HAMBURG,HHLA Container Terminal Burchardkai,CMA CGM TANGER,2020-01-03 20:20:00.000,9966,1118,147.80,23.28
3,ROTTERDAM WAALHAVEN,RST Waalhaven,2020-01-01 16:05:00.000,HAMBURG,HHLA Container Terminal Burchardkai,NIEVES B,2020-01-03 20:28:00.000,10318,1036,151.72,23.40
4,GDYNIA,Gdynia Container Terminal,2020-01-01 14:45:00.000,HAMBURG,HHLA Container Terminal Burchardkai,JUDITH,2020-01-04 05:05:00.000,16023,1440,170.02,25.19


In order to make sense of the markets team claim of the historical importance of each ship,
is to check if any of those ships have actually performed a "full circle" from HAMBURG/LA to HAMBURG/LA

In [4]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12054 entries, 0 to 12053
Data columns (total 11 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   last_port                    12054 non-null  object 
 1   last_terminal                12054 non-null  object 
 2   last_terminal_doc_timestamp  12054 non-null  object 
 3   current_port                 12054 non-null  object 
 4   current_terminal             12054 non-null  object 
 5   shipname                     12054 non-null  object 
 6   doc_timestamp                12054 non-null  object 
 7   grt                          12054 non-null  int64  
 8   teu                          12054 non-null  int64  
 9   length                       12054 non-null  float64
 10  width                        12054 non-null  float64
dtypes: float64(2), int64(2), object(7)
memory usage: 1.0+ MB


In [5]:
last_ports = raw_data.loc[:, "last_port"].unique().tolist()

In [6]:
current_port = raw_data.loc[:, "current_port"].unique().tolist()

In [7]:
print(f"current_ports: {len(current_port)} last_ports: {len(last_ports)}")

current_ports: 2 last_ports: 165


In [8]:
set(current_port).intersection(last_ports)

set()

the empty set here signifies that there are no roundtrips of the ships.

### lets investigate the timeline of some ships

In [9]:
print(f"the unique ships in this dataset are {len(raw_data.shipname.unique())}")

the unique ships in this dataset are 1397


In [10]:
ships_gone_both = []

In [11]:
for ship in raw_data.shipname.unique():
    dest = len(raw_data.loc[raw_data["shipname"]==ship,"current_port"].unique())
    if dest !=1:
        ships_gone_both.append(ship)

In [12]:
print(f"{len(ships_gone_both)} have gone to both ports.")

101 have gone to both ports.


In [14]:
user_defined = 2
for ship_no,(name,group) in enumerate(raw_data.groupby("shipname")):
    print(f"ship: {name}")
    display(group.sort_values(by="last_terminal_doc_timestamp"))
    if ship_no == user_defined:
        break

ship: A DAISEN


,last_port,last_terminal,last_terminal_doc_timestamp,current_port,current_terminal,shipname,doc_timestamp,grt,teu,length,width
7497,SHANGHAI,Shanghai Mingdong Container Terminal,2021-09-08 06:13:00.000,LOS ANGELES,Everport Services Terminal,A DAISEN,2021-10-07 11:55:00.000,18326,1740,175.46,27.69
9058,QINGDAO,QQCT,2022-01-12 07:56:00.000,LOS ANGELES,TraPac Terminal,A DAISEN,2022-02-17 11:00:00.000,18326,1740,175.46,27.69
9666,NINGBO,Beilun International Container Terminals,2022-03-22 14:58:00.000,LOS ANGELES,TraPac Terminal,A DAISEN,2022-04-12 10:23:00.000,18326,1740,175.46,27.69
10272,NINGBO,Beilun International Container Terminals,2022-05-13 15:25:00.000,LOS ANGELES,Yusen Terminal Inc.,A DAISEN,2022-06-03 22:19:00.000,18326,1740,175.46,27.69


ship: A LA MARINE


,last_port,last_terminal,last_terminal_doc_timestamp,current_port,current_terminal,shipname,doc_timestamp,grt,teu,length,width
122,ST PETERSBURG,Container Terminal Saint-Petersburg,2020-01-06 15:48:00.000,HAMBURG,Eurogate Container Terminal Hamburg,A LA MARINE,2020-01-14 22:49:00.000,16023,1440,170.06,25.2
287,ST PETERSBURG,Container Terminal Saint-Petersburg,2020-01-22 23:14:00.000,HAMBURG,Eurogate Container Terminal Hamburg,A LA MARINE,2020-01-28 23:34:00.000,16023,1440,170.06,25.2
446,ST PETERSBURG,Bulk cargo Quay,2020-02-05 22:01:00.000,HAMBURG,Eurogate Container Terminal Hamburg,A LA MARINE,2020-02-12 01:00:00.000,16023,1440,170.06,25.2
586,ST PETERSBURG,Container Terminal Saint-Petersburg,2020-02-20 20:52:00.000,HAMBURG,Cruise Center Steinwerder,A LA MARINE,2020-02-25 11:22:00.000,16023,1440,170.06,25.2
771,ST PETERSBURG,Container Terminal Saint-Petersburg,2020-03-06 22:48:00.000,HAMBURG,HHLA Container Terminal Burchardkai,A LA MARINE,2020-03-12 10:42:00.000,16023,1440,170.06,25.2
917,BREMERHAVEN,EUROGATE CTB,2020-03-24 07:17:00.000,HAMBURG,HHLA Container Terminal Tollerort,A LA MARINE,2020-03-25 13:31:00.000,16023,1440,170.06,25.2
1034,BREMERHAVEN,North Sea Terminal,2020-04-05 02:03:00.000,HAMBURG,HHLA Container Terminal Burchardkai,A LA MARINE,2020-04-05 19:26:00.000,16023,1440,170.06,25.2
1085,BREMERHAVEN,EUROGATE CTB,2020-04-08 21:15:00.000,HAMBURG,Eurogate Container Terminal Hamburg,A LA MARINE,2020-04-09 12:05:00.000,16023,1440,170.06,25.2
1182,COPENHAGEN,CMP CONTAINER TERMINAL,2020-04-17 06:45:00.000,HAMBURG,HHLA Container Terminal Altenwerder,A LA MARINE,2020-04-18 23:34:00.000,16023,1440,170.06,25.2
1264,BREMERHAVEN,EUROGATE CTB,2020-04-20 20:50:00.000,HAMBURG,Eurogate Container Terminal Hamburg,A LA MARINE,2020-04-25 03:51:00.000,16023,1440,170.06,25.2


ship: A.IDEFIX


,last_port,last_terminal,last_terminal_doc_timestamp,current_port,current_terminal,shipname,doc_timestamp,grt,teu,length,width
12036,NINGBO,Daxie China Merchants ICT,2022-11-05 05:47:00.000,LOS ANGELES,TraPac Terminal,A.IDEFIX,2022-11-23 12:04:00.000,18263,1700,182.0,26.0


In [15]:
terminal_ham = raw_data.loc[raw_data["current_port"]=="HAMBURG", "current_terminal"].unique().tolist()

In [16]:
terminal_la = raw_data.loc[raw_data["current_port"]=="LOS ANGELES", "current_terminal"].unique().tolist()

Essentially these here are our labels.

In [17]:
set(terminal_ham).intersection(set(terminal_la)),set(terminal_la).intersection(set(terminal_ham))

(set(), set())

In [18]:
print("possible final terminals", len(terminal_ham)+len(terminal_la))

possible final terminals 19


In [ ]:
raw_data["port_terminal"] = raw_data.apply(lambda x: f"{x['current_port']}_{x['current_terminal']}",axis=1)

In [ ]:
print(raw_data["port_terminal"].unique().shape[0])

In [20]:
ham_terms = raw_data.loc[raw_data["current_port"]=="HAMBURG", "current_terminal"].unique()

In [21]:
la_terms = raw_data.loc[raw_data["current_port"]=="LOS ANGELES", "current_terminal"].unique()

In [24]:
print(f"hamburg terminals {len(ham_terms)}, LA terminals {len(la_terms)}")

hamburg terminals 11 LA terminals 8


In [25]:
raw_data.apply(lambda x: f"{x['last_port']}_{x['last_terminal']}",axis=1).unique().shape

(269,)

if we keep in mind the port and the last terminal in mind we will have a severe case of sparse matrix
since we're going to have to create one hot representation for that feature.

# first attempt (baseline)

In [26]:
raw_data

,last_port,last_terminal,last_terminal_doc_timestamp,current_port,current_terminal,shipname,doc_timestamp,grt,teu,length,width
0,BREMERHAVEN,North Sea Terminal,2020-01-02 16:12:00.000,HAMBURG,Eurogate Container Terminal Hamburg,HEINRICH EHLER,2020-01-03 06:20:00.000,17488,1421,168.11,26.80
1,TILBURY,London Container Terminal,2020-01-01 21:56:00.000,HAMBURG,C. Steinweg Multipurpose Terminal,HENNEKE RAMBOW,2020-01-03 07:45:00.000,9981,868,134.40,22.74
2,ROTTERDAM MAASVLAKTE,Rotterdam World Gateway Terminal,2020-01-02 12:06:00.000,HAMBURG,HHLA Container Terminal Burchardkai,CMA CGM TANGER,2020-01-03 20:20:00.000,9966,1118,147.80,23.28
3,ROTTERDAM WAALHAVEN,RST Waalhaven,2020-01-01 16:05:00.000,HAMBURG,HHLA Container Terminal Burchardkai,NIEVES B,2020-01-03 20:28:00.000,10318,1036,151.72,23.40
4,GDYNIA,Gdynia Container Terminal,2020-01-01 14:45:00.000,HAMBURG,HHLA Container Terminal Burchardkai,JUDITH,2020-01-04 05:05:00.000,16023,1440,170.02,25.19
...,...,...,...,...,...,...,...,...,...,...,...
12049,ANTWERP,MSC PSA Delwaidedok Terminal,2022-11-21 18:24:00.000,HAMBURG,Eurogate Container Terminal Hamburg,CMA CGM LOME,2022-11-24 14:20:00.000,39753,3100,228.15,32.25
12050,ROTTERDAM MAASVLAKTE,APM Maasvlakte 2,2022-11-22 16:16:00.000,HAMBURG,Eurogate Container Terminal Hamburg,SAFMARINE BENGUELA,2022-11-24 20:38:00.000,35835,3078,223.30,32.20
12051,BREMERHAVEN,MSC Gate,2022-11-22 04:50:00.000,HAMBURG,HHLA Container Terminal Altenwerder,CONTAINERSHIPS STELLAR,2022-11-24 21:20:00.000,17982,1380,169.95,26.90
12052,VUOSAARI,Vuosaari Harbour,2022-11-21 03:41:00.000,HAMBURG,Eurogate Container Terminal Hamburg,ELBWIND,2022-11-24 22:56:00.000,10585,1000,151.72,23.40


In [ ]:
raw_data.columns

In [ ]:
kept_cols =['last_terminal_doc_timestamp', 'current_port', 'current_terminal', 'shipname', 'doc_timestamp', 'grt','teu', 'length', 'width', 'port_terminal']

In [ ]:
# distance could potentially be a much greater feature than everything else?

In [ ]:
kept_cols.append("diff")

In [ ]:
raw_data["diff"] = (pd.to_datetime(raw_data.loc[:, "doc_timestamp"]) - pd.to_datetime(raw_data.loc[:, "last_terminal_doc_timestamp"]))

In [ ]:
raw_data = raw_data.loc[:, kept_cols]

In [ ]:
raw_data = raw_data.loc[:, ["grt","teu","length", "width", "diff", "port_terminal"]]

In [ ]:
raw_data.describe()

In [ ]:
raw_data["diff"] = raw_data.apply(lambda x: x['diff'].seconds,axis=1)

In [ ]:
raw_data

# first simple baseline

In [ ]:
encoder = LabelEncoder()
encoder.fit(raw_data.loc[:,["port_terminal"]].values.reshape(-1,1))

In [ ]:
# since we actually do know the destination port, and those two ports do have different
# number of

In [ ]:
model = LogisticRegression()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(raw_data.loc[:, raw_data.columns!="port_terminal"], raw_data.loc[:,["port_terminal"]], test_size=0.2, random_state=42)

In [ ]:
y_train = encoder.transform(y_train)
# y_train = np.argmax(y_train, axis=1)
y_train = pd.DataFrame(y_train,columns=["label"])
y_test = encoder.transform(y_test)
# y_test = np.argmax(y_test, axis=1)
y_test = pd.DataFrame(y_test,columns=["label"])

In [ ]:
model.fit(X_train,y_train)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
print(f1_score(y_test, y_pred, average="micro"))

In [ ]:
print(f1_score(y_test, y_pred, average="macro"))

In [ ]:
accuracy_score(y_test, y_pred)

In [ ]:
model = SVC()
model.fit(X_train, y_train.values.ravel())

y_pred = model.predict(X_test)

print(f1_score(y_test, y_pred, average="micro"))
print(f1_score(y_test, y_pred, average="macro"))

In [ ]:
model = RandomForestClassifier()

In [ ]:
model.fit(X_train,y_train)
y_pred = model.predict(X_test)

In [ ]:
print(f1_score(y_test, y_pred, average="micro"))

In [ ]:
print(f1_score(y_test, y_pred, average="macro"))

In [ ]:
train_scaler = RobustScaler()

In [ ]:
X_train = train_scaler.fit_transform(X_train)

In [ ]:
X_train

In [ ]:
eval_scaler = RobustScaler()

In [ ]:
X_test = eval_scaler.fit_transform(X_test)

In [ ]:
model = RandomForestClassifier()

In [ ]:
model.fit(X_train, y_train.values.ravel())

y_pred = model.predict(X_test)

In [ ]:
print(f1_score(y_test, y_pred, average="micro"))

In [ ]:
print(f1_score(y_test, y_pred, average="macro"))

In [ ]:
model = LogisticRegression()
model.fit(X_train, y_train.values.ravel())

y_pred = model.predict(X_test)

print(f1_score(y_test, y_pred, average="micro"))
print(f1_score(y_test, y_pred, average="macro"))

### Enstablish simple baselines

### Can we do better?

# Possible future work

* Given the accuracy of your model, what would be the next steps in terms of deploying a solution such that the model’s value to the business is maximised?